In [2]:
import csv
import os
import re
from lxml import etree
import requests

In [2]:
def remove_html(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [3]:
filedirectory ='/Users/qad/gms'
os.chdir(filedirectory)

In [ ]:
#0 - Title
#1 - ConfirmedFilename
#2 - OldPath
#3 - TextIcon (old image file name)
#4 - Author 
#5 - Collection
#6 - Credits
#7 - Further reading
#8 - Genre
#9 - Introduction by
#10 - Introduction to the Author
#11 - Introduction to the Genre
#12 - Introduction to the Source
#13 - Introduction to the Text
#14 - Keywords
#15 - Language
#16 - Original language title
#17 - Period
#18 - TEI
#19 - Translator
#20 - Translator for citation
#21 - Year
#22 - Image caption
#23 - PURL

In [89]:
#Updated, use this one
#Open the source spreadsheet
with open('/Users/qad/gms/gms_october.tsv', 'r') as sourcefile:
    #Read the source spreadsheet
    #This spreadsheet uses tabs (\t) to separate columns
    #If you want to use a CSV, change the delimiter below to a comma
    sourcereader = csv.reader(sourcefile, delimiter="\t")
    #Skips the first row because it's the header row
    fields = next(sourcereader)
    #For each row in the spreadsheet
    for row in sourcereader:
        #Takes the value of the second column
        #which will be the base name for all things related to the text
        #and adds the markdown file extension
        newname = row[1]
        pagename = newname + '.md'
        
        #Opens a blank file with the filename you just created
        with open(pagename, 'w') as out:
            #Takes the value from column 4 where
            #we have the image filename for the text
            oldimagename = row[3]
            #Extracts from the image filename the suffix
            #that indicates what file type it is
            oldimagesuffix = re.sub(r'.*\.', '.', oldimagename)
            #Combines the new name for things related to the text
            #which we used above for the markdown name
            #and combines it with the filetype suffix for the image
            newimagename = newname + oldimagesuffix
            #Generates a name for the thumbnail file
            newthumb = newname + '-thumb' + oldimagesuffix
            
            
            #r = requests.get(row[14])
            #string_xml = r.content
            #print(string_xml)
            
            #Writes the dashes that indicate the markdwon header
            #Also writes out the layout and sidebar parameters
            #The \n will write a new line
            out.write('---\nlayout: text\nsidebar: left\n')
            #Creates a page title that combines row 0 (English title)
            #a pipe character, then the original language title
            #then a new line
            out.write("title: |\n  " + row[0] + ' | ' + row[16] + "\n")
            out.write("engtitle: |\n  " + row[0] + "\n")
            out.write("origtitle: |\n  " + row[16] + "\n")
            #Writes out the breadcrumb parameter
            out.write('breadcrumb: true\n')
            #Writes out a path to the permanent URL of the page
            out.write('permalink: "text/' + newname + '"\n')
            #Sets up a redirect from the old, often messy, URL
            out.write('redirect_from: ' + row[2] + '\n')
            #Defines the page name as the identifier for the text
            out.write('identifier: ' + pagename + '\n')
            #Writes out the pointer to the TEI file
            out.write('tei: /assets/tei/' + row[18] + '\n')
            #Writers out the pointer to the PDF file
            out.write('pdf: /assets/pdf/' + newname + '.pdf\n')
            #Writes out the text author
            out.write('textauthor: ' + row[4] + '\n')
            #Writes out the text language. There can be multiple languages
            #so we do this as a list in order for it to be parsed as tags
            out.write('languages: [' + row[15] + ']\n')
            #Writes out the period also as a list
            out.write('periods: [' + row[17] + ']\n')
            #Writes out the text collections as a list
            out.write('textcollections: [' + row[5] + ']\n')
            #Writes out the link to the digital repository
            out.write('sdr: https://purl.stanford.edu/druid \n')
            #Writes out the image filename
            out.write('image: /assets/img/text/' + newimagename + '\n')
            #Writes out the thumbnail image filename
            out.write('thumb: /assets/img/text/' + newthumb + '\n')
            #Writes out the image caption
            out.write("imagesource: |\n  " + row[22] + "\n")
            #Gets the name of the TEI file
            xmlname = (row[18])
            #Replaces XML with TXT since we've already extracted
            #The full plain text of the TEI w/o any tags
            textname = xmlname.replace('.xml','.txt')
            #Path to where the plain text versions of the texts are
            textpath = '/Users/qad/gms/vm_tei/' + textname
            #Prints the current file it's looking at
            #Just so we can keep track of what it's doing
            print(textpath)
            #Opens the text file
            with open(textpath, 'r') as fulltextfile:
                #Reads the full text
                fulltext = fulltextfile.read()
                #Writes the full text to the markdown header
                #Using the pipe/escape-all-things syntax
                #This is so the full text can be indexed by Jekyll for searching
                #Even though we're not displaying it on the page anywhere
                out.write("fulltext: |\n  "+ fulltext + '\n')
            #Writes out the end of the frontmatter 
            out.write('--- \n')
            
            #If there's actual content in the cell
            if len(row[12]) > 10:
                #Write out a subheader for intro to the source
                out.write('## Introduction to the Source \n')
                #Write out the contents of the cell
                out.write(row[12] + '\n')
            if len(row[13]) > 10:
                #Write out a subheader for intro to the text
                out.write('## Introduction to the Text \n')
                #Write out the contents of the cell
                out.write(row[13] + '\n')
            #Do the same for further reading
            if len(row[7]) > 10:
                out.write('## Further Reading \n')
                out.write(row[7] + '\n')
            
            out.write('## Credits\n' + row[6] + '\n')
            #out.write('## Suggested Citation\n' + row[17])

/Users/qad/gms/vm_tei/efrosin_tale_dracula.txt
/Users/qad/gms/vm_tei/roman_origins_lithuanians.txt
/Users/qad/gms/vm_tei/miracles_vladimir_mother_god.txt
/Users/qad/gms/vm_tei/efrosin_solomon_kitovras.txt
/Users/qad/gms/vm_tei/su_shi_sending_shugu_west_lake.txt
/Users/qad/gms/vm_tei/li_qingzhao_looking_searching.txt
/Users/qad/gms/vm_tei/li_qingzhao_setting_sun.txt
/Users/qad/gms/vm_tei/li_qingzhao_wind_stops_dust_fragrant.txt
/Users/qad/gms/vm_tei/li_qingzhao_wind_gentle_sunlight_soft.txt
/Users/qad/gms/vm_tei/li_qingzhao_sky_merged.txt
/Users/qad/gms/vm_tei/li_qingzhao_at_night.txt
/Users/qad/gms/vm_tei/li_qingzhao_courtyard_deep.txt
/Users/qad/gms/vm_tei/li_qingzhao_who_planted_banana_trees.txt
/Users/qad/gms/vm_tei/li_qingzhao_outside_window_spring_color.txt
/Users/qad/gms/vm_tei/li_qingzhao_incense_chills.txt
/Users/qad/gms/vm_tei/li_qingzhao_in_thin_fog.txt
/Users/qad/gms/vm_tei/li_qingzhao_often_call.txt
/Users/qad/gms/vm_tei/li_qingzhao_wind_comes_from_lake.txt
/Users/qad/gms/v

/Users/qad/gms/vm_tei/yan_jidao_colorful_sleeves.txt
/Users/qad/gms/vm_tei/yan_jidao_dew_metal_palm.txt
/Users/qad/gms/vm_tei/yan_jidao_holding_fragrant_paper.txt
/Users/qad/gms/vm_tei/yan_jidao_scene_painted_screen.txt
/Users/qad/gms/vm_tei/yan_jidao_when_wake_up.txt
/Users/qad/gms/vm_tei/yan_shu_chrysanthemums.txt
/Users/qad/gms/vm_tei/yan_shu_water_pond.txt


# Create tag pages

In [ ]:
tagdirectory ='/Users/qad/gms/tags'
os.chdir(tagdirectory)

In [ ]:
#languages = ['Aljamiado', 'Arabic', 'Asia', 'Catalan', 'Chinese', 'Eastern Europe', 'English', 'French', 'German', 'Greek', 'Hebrew', 'Italian', 'Japanese', 'Latin', 'Middle East and North Africa', 'Occitan', 'Persian', 'Portuguese', 'Spanish', 'Syriac', 'The Americas', 'Welsh', 'Western Europe']

In [ ]:
#keywords
#languages = ['1025', '1050', '1066', '1100', '117', '1185', '1270', '1279', '1300', '138', '1400', '1500', '1600', '16th century', '1700', '449', '500', '627', '750', '960', 'adultery', 'Advice in literature', 'Aging', 'Alliteration', 'Anecdotes', 'Anglo', 'animals', 'Apocalyptic literature', 'Apostles', 'Arabic', 'Archery', 'Art', 'Arthurian romances', 'autumn', 'Avarice', 'Basil II Bulgaroctonus 976', 'Battle of the', 'Bible', 'Biography', 'Blessed Virgin', 'Body and soul in literature', 'burial', 'Byzantine Empire', 'Caliphs', 'Calligraphy', 'calligraphy', 'Calvinism', 'Canzonets (Part songs)', 'Catalan poetry', 'chastity', 'China', 'Chinese', 'Chinese literature', 'Chinese poetry', 'Christian drama', 'Christian Poetry', 'Christianity', 'Church history', 'Classical Arabic literature', 'Classical period', 'Clouds in literature', 'Codes (Jewish law)', 'Concrete poetry', 'cosmology', 'Criticism interpretation etc. Jewish', 'Crusades', 'Crusades and art', 'Dance', 'death', 'Debate poetry', 'Deluge', 'Deluge (Islam)', 'devil', 'Devotional literature', 'Dialogues', 'Didactic literature', 'Didactic poetry', 'Discipline', 'Ditch', 'Divination', 'dogs', 'Dragons', 'Dreams', 'Drinking of alcoholic beverages in literature', 'Early works to 1800', 'education', 'Eighth', 'Enemies', 'Epic literature', 'erotic literature', 'etc', 'Fables', 'Fabliaux', 'faith', 'farce', 'Feudalism', 'fire', 'Flanders', 'flowers', 'Folk songs', 'France', 'Franks', 'French language', 'French literature', 'friendship', 'garlic', 'gastronomy', 'Gaul', 'Gaze in literature', 'gender', 'Gender identity in literature', 'German', 'German (Middle High)', 'German literature', 'German poetry', 'Germany', 'Ghost stories', 'God', 'Goths', 'Grace (Theology)', 'Great Britain', 'Gu ci', 'Gulf of (Tunisia)', 'Hadrian', 'Hagiography', 'Heaven', 'Hell', 'Hermits', 'Hero (Greek mythology)', 'History', 'history', 'History and criticism', 'history—humor', 'humor', 'Hymns', 'Illumination of books and manuscripts', 'incest', 'invective', 'inventions', 'Islam', 'Islamic poetry', 'Italian poetry', 'Japanese literature', 'Japanese poetry', 'justice', 'Justification (Christian theology)—history of doctrines', 'Kings', 'Kings and rulers', 'Laments', 'Latin (Medieval and modern)', 'Latin (medieval)', 'Latin Medieval and modern', 'Latin poetry', 'Laudatory poetry', 'Law commentaries', 'Legends', 'leopard', 'liturgy', 'local history', 'loneliness', 'Lord’s Supper', 'love', 'Love songs', 'Low German drama', 'lust', 'Lyric poetry', 'manuscript', 'marriage', 'martyrdom', 'Mary', 'Master and servant', 'Medieval', 'Medieval and modern', 'medieval poetry', 'Medina', 'Melancholy', 'Melodrama', 'Men', 'Metaphor', 'Middle French', 'Middle High German', 'Middle Welsh', 'misogyny', 'Monasteries', 'Monastic and religious life', 'Monasticism and religious orders', 'Monks', 'Monsters in literature', 'moon', 'Mothers', 'music', 'Music', 'Mysticism', 'Mythology', 'narrative poetry', 'nature', 'Newlyweds', 'Nuns’ writings', 'obscene words', 'Occitan', 'Old High German', 'Old High German songs', 'Oneiromancy', 'Oracles', 'parody', 'Pastoral care', 'Patriarchs and patriarchate', 'Persian', 'Peter', 'plots', 'Poetry', 'Political poetry', 'Politics and war', 'Popes', 'Portuguese literature', 'prayer', 'Prayer', 'Prayers and Devotions', 'Prayers and devotions', 'Precious stones', 'Prefaces', 'Pregnancy', 'Printing presses', 'progress', 'Prophecies', 'Prostitutes in literature', 'Protestant Reformation', 'punishment', 'Qasidas', 'queens', 'Recluses', 'Religious drama', 'religious literature', 'Religious Poetry', 'Renaissance', 'Rhine River Valley', 'Riddles', 'ritual', 'Roland (Legendary character)', 'Romances', 'Romans', 'Romans in literature', 'Rome', 'rulers', 'Saint', 'Saints', 'satire', 'Saturn', 'Saturn (Planet)', 'Saudi Arabia', 'Saxon period', 'Sermons', 'sex', 'sin', 'Singing', 'Song dynasty', 'soul', 'Soups', 'sovereignty', 'spiritual poetry', 'Stories', 'Strasbourg (France)', 'Sufi literature', 'sun', 'supernatural', 'Syriac', 'Syriac language', 'Syriac Poetry', 'Taoism', 'technology', 'temptation', 'TEST', 'Texts', 'the Apostle', 'time', 'To 1400', 'To 1500', 'To 843', 'To 987', 'torture', 'Tournaments', 'Translations into English', 'Travel', 'Troy (Extinct city)', 'Tunis', 'Unicorns', 'Vandals', 'Vikings', 'violence', 'virtue', 'Visions in literature', 'wager of battle', 'Waka', 'war', 'water', 'weather', 'Welsh language', 'wine', 'wisdom', 'Wisdom literature', 'Wit and humor', 'women']

In [ ]:
#period
languages = ['15th Century', '11th Century', '14th Century', '13th Century', '12th Century', '9th Century', '8th Century', '7th Century', '10th Century', '16th Century']

In [ ]:
#genre
languages = ['Canso', 'Chronicles', 'Devotional Prose', 'Devotional Verse', 'Dialogue', 'Didactic Verse', 'Exemplum', 'Handbook', 'History', 'Humor', 'Law commentaries', 'Letter', 'Lyric Verse - Song', 'Narrative Prose', 'Narrative Verse', 'Parody', 'Play', 'Poetry', 'Riddle', 'Satire', 'Tanka', 'Verse Epic', 'Vita']

In [ ]:
for language in languages:
    languageclean = language.lower()
    languageclean = languageclean.replace(' ', '_')
    newfilename = lower.replace(' ', '_') + '.md'
    print(newfilename)
    with open(newfilename, 'w') as out:
        out.write('---\n')
        out.write('layout: genrepage\n')
        out.write('title: '+ language + '\n')
        out.write('identifier: '+ languageclean + '\n')
        out.write('permalink: /regions/' + languageclean)
        out.write('---')

## TEI-free texts

In [ ]:
filedirectory ='/Users/qad/gms/vm_tei'
os.chdir(filedirectory)

In [ ]:
for filename in os.listdir(filedirectory):
    textlines = []
    if filename.endswith('.xml'):
        textname = filename.replace('.xml', '.txt')
        with open(filename, 'r') as xmlfile:
            with open(textname, 'w') as textfile:
                xmlitems = xmlfile.readlines()
                for xmlitem in xmlitems:
                    if '<lem' in xmlitem:
                        cleanlem = remove_html(xmlitem)
                        cleanlem = cleanlem.replace('\t', '')
                        cleanlem = cleanlem.replace('\n', '')
                        textlines.append(cleanlem)
                    if '<rdg' in xmlitem:
                        cleanrdg = remove_html(xmlitem)
                        cleanrdg = cleanrdg.replace('\t', '')
                        cleanrdg = cleanrdg.replace('\n', '')
                        textlines.append(cleanrdg)
                for textline in textlines:
                    textstring += ' ' + textline
                    textfile.write(textline)
                    textfile.write(' ')
                textlines = []

## Image renamer

In [8]:
os.chdir('/Users/qad/gms/bigimages')
with open('/Users/qad/gms/gms_static_oct28.tsv', 'r') as datafile:
    datareader = csv.reader(datafile, delimiter='\t')
    fields = next(datareader)
    for row in datareader:
        if row[3] != '':
            oldimagename = row[3]
            oldimagesuffix = re.sub(r'.*\.', '.', oldimagename)
            newname = row[1]
            fullnewname = '/Users/qad/gms/images/imagesclean/' + newname + oldimagesuffix
            #fullnewname = '/Users/qad/gms/images/imagesclean/' + newname + '-thumb' + oldimagesuffix
            fulloldname = '/Users/qad/gms/bigimages/' + oldimagename
            print(fullnewname)
            os.rename(oldimagename, fullnewname)

/Users/qad/gms/images/imagesclean/al_harith_toward_demise.png
/Users/qad/gms/images/imagesclean/al_isbahani_prayer_presence_king.jpg
/Users/qad/gms/images/imagesclean/al_mulhalhil_twilights_ruins.png
/Users/qad/gms/images/imagesclean/al_nuwayri_book_knowledge.jpg
/Users/qad/gms/images/imagesclean/al_shafii_let_days_do_they_please.jpg
/Users/qad/gms/images/imagesclean/al_shafii_were_it_not_that_poesy.jpg
/Users/qad/gms/images/imagesclean/al_shafii_your_friend_contests.png
/Users/qad/gms/images/imagesclean/alcuin_song_4_go_quickly.jpg
/Users/qad/gms/images/imagesclean/andre_romance_of_the_french.jpg
/Users/qad/gms/images/imagesclean/bar_ebroyo_on_april.JPG
/Users/qad/gms/images/imagesclean/bracciolini_conclusio.jpg
/Users/qad/gms/images/imagesclean/bracciolini_introduction.jpg
/Users/qad/gms/images/imagesclean/bracciolini_joke_114.jpg
/Users/qad/gms/images/imagesclean/bracciolini_joke_121.jpg
/Users/qad/gms/images/imagesclean/bracciolini_joke_13.png
/Users/qad/gms/images/imagesclean/brac